Multiome Data
Multiome gene expression (GEX)
Assay for Transposase-Accessible Chromatin (ATAC) datasets (ATAC)

# ATAC to GEX

In [2]:
import numpy as np
import pandas as pd
import logging
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import os
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt

In [7]:
multi_atac = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_train_mod1.h5ad")
multi_gex = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_train_mod2.h5ad")
multi_sol = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_train_sol.h5ad")

In [10]:
print(f"The training ATAC data has {multi_atac.n_obs} observations and {multi_atac.n_vars} features.")
print(f"The training GEX data has {multi_gex.n_obs} observations and {multi_gex.n_vars} features.")
print(f"The training solution data has shape {multi_sol.shape}.")

The training ATAC data has 16394 observations and 116490 features.
The training GEX data has 16394 observations and 13431 features.
The training solution data has shape (16394, 16394).


In [10]:
multi_atac_test = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_test_mod1.h5ad")
multi_gex_test = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_test_mod2.h5ad")
multi_sol_test = ad.read_h5ad("/Users/xcyan/Desktop/scot/data/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_test_sol.h5ad")

In [12]:
multi_gex_test.layers["counts"]

<6069x13431 sparse matrix of type '<class 'numpy.float64'>'
	with 8279241 stored elements in Compressed Sparse Column format>

In [11]:
print(f"The testing ATAC data has {multi_atac_test.n_obs} observations and {multi_atac_test.n_vars} features.")
print(f"The testing GEX data has {multi_gex_test.n_obs} observations and {multi_gex_test.n_vars} features.")
print(f"The testing solution data has shape {multi_sol_test.shape}.")

The testing ATAC data has 6069 observations and 116490 features.
The testing GEX data has 6069 observations and 13431 features.
The testing solution data has shape (6069, 6069).


In [13]:
multi_atac_train_array=multi_atac.X.toarray()
print(multi_atac_train_array.shape)
print(multi_atac_train_array[1])
print(multi_atac_train_array[1].shape)

(16394, 116490)
[0. 0. 0. ... 0. 0. 0.]
(116490,)


In [14]:
multi_gex_train_array=multi_gex.X.toarray()
print(multi_gex_train_array.shape)
print(multi_gex_train_array[1].shape)

(16394, 13431)
(13431,)


In [16]:
multi_gex.var

,gene_ids,feature_types
AL627309.5,ENSG00000241860,GEX
LINC01409,ENSG00000237491,GEX
LINC01128,ENSG00000228794,GEX
NOC2L,ENSG00000188976,GEX
KLHL17,ENSG00000187961,GEX
...,...,...
MT-ND5,ENSG00000198786,GEX
MT-ND6,ENSG00000198695,GEX
MT-CYB,ENSG00000198727,GEX
AL592183.1,ENSG00000273748,GEX


In [17]:
multi_atac.var

,feature_types
chr1-9776-10668,ATAC
chr1-180726-181005,ATAC
chr1-181117-181803,ATAC
chr1-191133-192055,ATAC
chr1-267562-268456,ATAC
...,...
GL000219.1-90062-90937,ATAC
GL000219.1-99257-100160,ATAC
KI270726.1-27152-28034,ATAC
KI270713.1-21434-22336,ATAC


In [18]:
multi_gex.uns['dataset_id'] 
multi_adt.uns['dataset_id'] 

'openproblems_bmmc_multiome_phase1_MM_atac2gex'

In [19]:
multi_sol_array=multi_sol.X.toarray()
print(multi_sol_array.shape)
print(multi_sol_array[1,1:20])

(16394, 16394)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
multi_sol_test_array=multi_sol_test.X.toarray()
print(multi_sol_test_array.shape)
print(multi_sol_test_array[1,1:20])

(6069, 6069)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
multi_sol.obs

,batch
TAGTTGTCACCCTCAC-1-s1d1,s1d1
CTATGGCCATAACGGG-1-s1d1,s1d1
CCGCACACAGGTTAAA-1-s1d1,s1d1
TCATTTGGTAATGGAA-1-s1d1,s1d1
ACCACATAGGTGTCCA-1-s1d1,s1d1
...,...
TAGTAAGCAACTAGGG-8-s3d6,s3d6
TGGTCCTTCGGCTAGC-8-s3d6,s3d6
CGCTTGCGTTGTTGGA-8-s3d6,s3d6
ACCCTCCCAGCCAGTT-8-s3d6,s3d6


In [22]:
multi_sol_test.obs

,batch
CTACCGGCAGTTGCGT-2-s1d2,s1d2
CCGCTAAAGGACCGCT-2-s1d2,s1d2
CTAACCACAGTTTGGC-2-s1d2,s1d2
CGAAGAATCGATTCTT-2-s1d2,s1d2
CTTTGTCCATACCCGG-2-s1d2,s1d2
...,...
CGCGATTCATCATGTG-2-s1d2,s1d2
AAGCATGAGCATGGTT-2-s1d2,s1d2
GAGAACCAGCAGGTGG-2-s1d2,s1d2
AGCTTCCTCCCTGGTT-2-s1d2,s1d2


# GEX TO ATAC

In [23]:
rna_atac = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_train_mod1.h5ad")
rna_gex = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_train_mod2.h5ad")
rna_sol = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_train_sol.h5ad")

In [31]:
print(f"The training GEX data has {rna_atac.n_obs} observations and {rna_atac.n_vars} features.")
print(f"The training ATAC data has {rna_gex.n_obs} observations and {rna_gex.n_vars} features.")
print(f"The training solution data has shape {rna_sol.shape}.")

The training GEX data has 16394 observations and 13431 features.
The training ATAC data has 16394 observations and 116490 features.
The training solution data has shape (16394, 16394).


In [29]:
rna_gex_test = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_test_mod1.h5ad")
rna_atac_test = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_test_mod2.h5ad")
rna_sol_test = ad.read_h5ad("/Users/xcyan/Desktop/2952G_Project/starter_kit-match_modality-python/output/datasets/match_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_test_sol.h5ad")

In [30]:
print(f"The testing GEX data has {rna_gex_test.n_obs} observations and {rna_gex_test.n_vars} features.")
print(f"The testing ATAC data has {rna_atac_test.n_obs} observations and {rna_atac_test.n_vars} features.")
print(f"The testing solution data has shape {rna_sol_test.shape}.")

The testing GEX data has 6069 observations and 13431 features.
The testing ATAC data has 6069 observations and 116490 features.
The testing solution data has shape (6069, 6069).


In [33]:
rna_gex_array=rna_gex.X.toarray()
print(rna_gex_array.shape)
print(rna_gex_array[1])

(16394, 116490)
[0. 0. 0. ... 0. 0. 0.]


In [34]:
rna_gex.uns['dataset_id']

'openproblems_bmmc_multiome_phase1_MM_gex2atac'